# SQL vs NoSQL

## Introduction

### SQL

An SQL database contains data organized in tables. These tables have a predefined structure, much like a data frame, in the sense that variables are represented by columns and individual data points are represented as rows:

<p align="left">
<img src="SQL.png" style="width: 600px;"/>
</p>
<small>Source: BUCKLER, Craig: "SQL vs NoSQL: The Differences", 2015 on SitePoint.com</small>


### NoSQL

NoSQL data on the other hand are organized in nodes, much like a tree structure. This differs significantly from SQL tables since data points do not need to contain the same variables as others. 

<p align="left">
<img src="NoSQL.png" style="width: 600px;"/>
</p>
<small>Source: BUCKLER, Craig: "SQL vs NoSQL: The Differences", 2015 on SitePoint.com</small>


In this aspect NoSQL is more versatile: If extra information for a certain element is available, it can be immediately added to the database. However, if this is not done cautiously, the lack of structure in the data can make analysis and maintenance very difficult. SQL's requirement of formal definitions for tables can be seen an advantage to avoid this situation.

## Case Study: Designing a CRM database

### The Application

To illustrate the differences in using SQL and NoSQL, we will design a database tool to keep track of student presentation topics and their contact information using both database paradigms.

Basic functionality requires keeping track of:

1. Students
2. Contact info
3. Each student's presentation topics

To begin, let's start looking at the raw data we have available:

In [1]:
students = ['Aysja Johnson',
            'Gabriel Augusto Sanchez',
            'Ivan Dimitrov',
            'Jimin Yu',
            'Karen Leung',
            'Kurt Brown',
            'Matt Bartholomew',
            'Nathan Helm Burger',
            'Rafael Lacerda',
            'Zi (Russ) Gu',
            'Sven Chilton',
            'Tegan McCaslin'
           ]

# The contacts list is ordered by their corresponding student.
contacts = ['aysjajohnson@gmail.com',
            'gebsanchez@gmail.com',
            'ivan@emailio.com',
            'jy1003@nyu.edu',
            'kleung92@gmail.com',
            'kurtbrown137@gmail.com',
            'bartholomatt@gmail.com',
            'nathan.helm.burger@gmail.com',
            'rafaelplmelo@gmail.com',
            'rustygu@gmail.com',
            'sven.chilton@gmail.com',
            'tegan.mccaslin@gmail.com'
]

# The subjects list is ordered by their corresponding student.
subjects = [['Semi-supervised learning'],
            ['Time series analysis'],
            ['Network analysis', 'Social networks'],
            ['Association rules'],
            ['Shiny'],
            ['Bayesian methods', 'Multi-level modeling', 'Hierarchical methods'],
            ['Apache Spark'],
            ['MapReduce', 'Apache Hadoop'],
            ['SQL vs. NoSQL'],
            ['Multiple hypothesis testing', 'Bonferroni correction', 'A/B testing', 'Multi-armed bandit'],
            ['Support vector machines'],
            ['Markov models', 'Hidden Markov models']
           ]

### Interacting with the database

Simple outline:
1. Initialize the database
2. Create entity that will hold our data
3. Insert the data
4. Retrieve the data

For this tutorial, we will use simple database systems that store the data locally, as opposed to more common and robust database technologies that rely on servers, authentication and networking.

#### SQL
Let's start off with SQL, using the SQLite package:

In [19]:
"""
Database Initialization
"""

# Using sqlite3 to store and retrieve data.
import sqlite3 as sql
# Using pandas to have dataframe functionality
import pandas as pd

# This also creates a database file if it doesn't already exist.
connection = sql.connect("sqlite.db")
# This is a standard precaution to avoid database locks - 30s
connection.execute("PRAGMA busy_timeout = 30000;")
connection.execute("PRAGMA foreign_keys = ON;")


# This is the main object we will use to execute database commands
# and retrieve data.
cursor = connection.cursor()

In [3]:
"""
Creating data entity
"""

# If a table already exists when we attempt to create it, we will
# receive an error message. To avoid this, let's DROP any existing
# tables with the same name.
try:
    cursor.execute("DROP TABLE students;")
except:
    pass

# Define table structures and create them.
cursor.execute("""
                CREATE TABLE students(
                id INTEGER PRIMARY KEY, name TEXT, email TEXT
                );
                """)

In [4]:
"""
Insert data to our table
"""

cursor.executemany("""
                INSERT INTO students(name, email)
                VALUES(?, ?);
                """, zip(students, contacts))

In [5]:
"""
Data retrieval
"""

# To retrieve this information from the database, first run the query:
cursor.execute("""
                SELECT *
                FROM students;
                """)
# The data is now accessible via the cursor:
sql_students = cursor.fetchall()
sql_students_df = pd.DataFrame(data=sql_students, columns=[i[0] for i in cursor.description])
sql_students_df

id                     name                         email
0    1            Aysja Johnson        aysjajohnson@gmail.com
1    2  Gabriel Augusto Sanchez          gebsanchez@gmail.com
2    3            Ivan Dimitrov              ivan@emailio.com
3    4                 Jimin Yu                jy1003@nyu.edu
4    5              Karen Leung            kleung92@gmail.com
5    6               Kurt Brown        kurtbrown137@gmail.com
6    7         Matt Bartholomew        bartholomatt@gmail.com
7    8       Nathan Helm Burger  nathan.helm.burger@gmail.com
8    9           Rafael Lacerda        rafaelplmelo@gmail.com
9   10             Zi (Russ) Gu             rustygu@gmail.com
10  11             Sven Chilton        sven.chilton@gmail.com
11  12           Tegan McCaslin      tegan.mccaslin@gmail.com

#### NoSQL

Let's try to to the same thing using an equivalent NoSQL system, such as UnQLite.

In [22]:
"""
Database Initialization
"""

from unqlite import UnQLite
db = UnQLite('nosql.db')

In [102]:
"""
Create data entity
"""
# We could use NoSQL as a Key/Value store, but the most salient
# feature of NoSQL is the ability to use JSON collections.
nosql_students = db.collection('users')
nosql_students.drop()
nosql_students.create()

In [103]:
"""
Insert data to our students entity
"""

student_json = [dict(zip(["name", "email"], row)) for row in zip(students, contacts)]

nosql_students.store(student_json)

11

In [104]:
"""
Data retrieval
"""
nosql_students_df = pd.DataFrame(nosql_students.all())
nosql_students_df

__id                         email                     name
0      0        aysjajohnson@gmail.com            Aysja Johnson
1      1          gebsanchez@gmail.com  Gabriel Augusto Sanchez
2      2              ivan@emailio.com            Ivan Dimitrov
3      3                jy1003@nyu.edu                 Jimin Yu
4      4            kleung92@gmail.com              Karen Leung
5      5        kurtbrown137@gmail.com               Kurt Brown
6      6        bartholomatt@gmail.com         Matt Bartholomew
7      7  nathan.helm.burger@gmail.com       Nathan Helm Burger
8      8        rafaelplmelo@gmail.com           Rafael Lacerda
9      9             rustygu@gmail.com             Zi (Russ) Gu
10    10        sven.chilton@gmail.com             Sven Chilton
11    11      tegan.mccaslin@gmail.com           Tegan McCaslin

### Differences so far

#### Ease of use
It may seem at first that NoSQL has a much simpler interface, but this is not necessarily true. It is trivial to set up an Object-Relational Mapping tool (ORM) such as SQL Alchemy that will provide a simple interface for several SQL and NoSQL databases, which would mostly let the user forget what kind of database is running on the backend.

#### Formalisms
Syntax aside, SQL also requires that tables be defined beforehand, detailing the exact variables to be used and their corresponding data types. On the other hand, not only does NoSQL let you define variables on-the-fly, it also allows for elements to have different variables!

Let's add a new student to our cohort who has extra contact information:

In [6]:
# SQL
cursor.execute("""
                ALTER TABLE students
                ADD COLUMN phone TEXT;
                """)
cursor.execute("""
                INSERT INTO students(name, email, phone)
                VALUES('Namey McNamerson',
                'mcnamerson@example.com',
                '555 555 5555');
                """)
cursor.execute("""
                SELECT *
                FROM students;
                """)
sql_students = cursor.fetchall()
sql_students_df = pd.DataFrame(data=sql_students, columns=[i[0] for i in cursor.description])
sql_students_df

id                     name                         email         phone
0    1            Aysja Johnson        aysjajohnson@gmail.com          None
1    2  Gabriel Augusto Sanchez          gebsanchez@gmail.com          None
2    3            Ivan Dimitrov              ivan@emailio.com          None
3    4                 Jimin Yu                jy1003@nyu.edu          None
4    5              Karen Leung            kleung92@gmail.com          None
5    6               Kurt Brown        kurtbrown137@gmail.com          None
6    7         Matt Bartholomew        bartholomatt@gmail.com          None
7    8       Nathan Helm Burger  nathan.helm.burger@gmail.com          None
8    9           Rafael Lacerda        rafaelplmelo@gmail.com          None
9   10             Zi (Russ) Gu             rustygu@gmail.com          None
10  11             Sven Chilton        sven.chilton@gmail.com          None
11  12           Tegan McCaslin      tegan.mccaslin@gmail.com          None
12  13         Namey McNamerson        mcnamerson@example.com  555 555 5555

In [105]:
# NoSQL
nosql_students.store({'name':'Namey McNamerson',
                      'email':'mcnamerson@example.com',
                      'phone':'555 555 5555'})

nosql_students_df = pd.DataFrame(nosql_students.all())
nosql_students_df

__id                         email                     name         phone
0      0        aysjajohnson@gmail.com            Aysja Johnson           NaN
1      1          gebsanchez@gmail.com  Gabriel Augusto Sanchez           NaN
2      2              ivan@emailio.com            Ivan Dimitrov           NaN
3      3                jy1003@nyu.edu                 Jimin Yu           NaN
4      4            kleung92@gmail.com              Karen Leung           NaN
5      5        kurtbrown137@gmail.com               Kurt Brown           NaN
6      6        bartholomatt@gmail.com         Matt Bartholomew           NaN
7      7  nathan.helm.burger@gmail.com       Nathan Helm Burger           NaN
8      8        rafaelplmelo@gmail.com           Rafael Lacerda           NaN
9      9             rustygu@gmail.com             Zi (Russ) Gu           NaN
10    10        sven.chilton@gmail.com             Sven Chilton           NaN
11    11      tegan.mccaslin@gmail.com           Tegan McCaslin           NaN
12    12        mcnamerson@example.com         Namey McNamerson  555 555 5555

NoSQL's flexibility is very apparent here. However, this must be used with care to avoid maintenance and analysis nightmares. Since there is no data validation at work here, phone numbers could be stored as either integers, text or many other things as we will see very soon.

Unclean and unstructured data is extra work, and it is good practice to implement rules in your application to ensure the database conforms to the planned model.

### Joins and Normalization

At this point, we run into a minor issue when we consider adding student's subjects in a SQL database: Since each student can have many subjects (known as a One-to-Many relationship), how should we store this in the database?

If we keep everything in the same table, we would have several rows for each student. This means:

- The ID column would no longer relate to students but for each subject
- We would have repeated information in several rows, which would unnecessarily increase database size
- Whenever we queried the database for a student's email, we would get several rows back, which creates inefficiencies for bandwidth and processing time:

In [9]:
data = [[(pair[0]+1, pair[1], pair[2],p) for p in pair[3]] for pair in zip(range(12), students, contacts, subjects)]
lst = list()
[lst.extend(d) for d in data]
df = pd.DataFrame(lst, columns=["id", "name", "email", "subject"])
df[df.id == 10]

id          name              email                      subject
13  10  Zi (Russ) Gu  rustygu@gmail.com  Multiple hypothesis testing
14  10  Zi (Russ) Gu  rustygu@gmail.com        Bonferroni correction
15  10  Zi (Russ) Gu  rustygu@gmail.com                  A/B testing
16  10  Zi (Russ) Gu  rustygu@gmail.com           Multi-armed bandit

Instead, we must turn to the concept of Normalization or reduction to Canonical Form: "The process of organizing the columns (attributes) and tables (relations) of a relational database to minimize data redundancy."<sup>1</sup>

Simply, we must store the Subject data in a separate table and create a relationship column to associate the subject to its student. Associating data between tables is done by creating a second ID column that has values that point to the other table's Primary Key and defining the column as a Foreign Key, to allow for SQL-side validation:

<sub>Source: Wikipedia, https://en.wikipedia.org/wiki/Database_normalization</sub>

In [22]:
cursor.execute("DROP TABLE subjects;")
cursor.execute("""
                CREATE TABLE subjects(
                id INTEGER PRIMARY KEY,
                subject TEXT,
                student_id INT,
                FOREIGN KEY(student_id) REFERENCES students(id)
                );
                """)

In [26]:
# Build our student foreign key numbers
from functools import reduce
rep = [len(d) for d in data]
student_ids = [i+1 for i in range(13)]
student_ids = [[i[0]]*i[1] for i in zip(student_ids, rep)]

# Flatten students
student_ids = reduce(lambda x,y: x+y, student_ids)
# Flatten subjects
subjects_flat = reduce(lambda x,y: x+y, subjects)

"""
Insert data to our table
"""

cursor.executemany("""
                INSERT INTO subjects(subject, student_id)
                VALUES(?, ?);
                """, zip(subjects_flat, student_ids))

cursor.execute("""
                SELECT a.name, a.email, b.subject
                FROM students AS a
                LEFT JOIN subjects AS b
                ON a.id == b.student_id
                ;
                """)

sql_subjects = cursor.fetchall()
sql_subjects_df = pd.DataFrame(data=sql_subjects, columns=[i[0] for i in cursor.description])
sql_subjects_df

name                         email  \
0             Aysja Johnson        aysjajohnson@gmail.com   
1   Gabriel Augusto Sanchez          gebsanchez@gmail.com   
2             Ivan Dimitrov              ivan@emailio.com   
3             Ivan Dimitrov              ivan@emailio.com   
4                  Jimin Yu                jy1003@nyu.edu   
5               Karen Leung            kleung92@gmail.com   
6                Kurt Brown        kurtbrown137@gmail.com   
7                Kurt Brown        kurtbrown137@gmail.com   
8                Kurt Brown        kurtbrown137@gmail.com   
9          Matt Bartholomew        bartholomatt@gmail.com   
10       Nathan Helm Burger  nathan.helm.burger@gmail.com   
11       Nathan Helm Burger  nathan.helm.burger@gmail.com   
12           Rafael Lacerda        rafaelplmelo@gmail.com   
13             Zi (Russ) Gu             rustygu@gmail.com   
14             Zi (Russ) Gu             rustygu@gmail.com   
15             Zi (Russ) Gu             rustygu@gmail.com   
16             Zi (Russ) Gu             rustygu@gmail.com   
17             Sven Chilton        sven.chilton@gmail.com   
18           Tegan McCaslin      tegan.mccaslin@gmail.com   
19           Tegan McCaslin      tegan.mccaslin@gmail.com   
20         Namey McNamerson        mcnamerson@example.com   

                        subject  
0      Semi-supervised learning  
1          Time series analysis  
2              Network analysis  
3               Social networks  
4             Association rules  
5                         Shiny  
6              Bayesian methods  
7          Hierarchical methods  
8          Multi-level modeling  
9                  Apache Spark  
10                Apache Hadoop  
11                    MapReduce  
12                SQL vs. NoSQL  
13                  A/B testing  
14        Bonferroni correction  
15           Multi-armed bandit  
16  Multiple hypothesis testing  
17      Support vector machines  
18         Hidden Markov models  
19                Markov models  
20                         None

So we can get to the same information if we wish, without storing redundant data.

NoSQL has different tradeoffs: We could choose to move the data to another entity named 'subjects' like we did in SQL, or, to take advantage of JSON collections, we can store extra data for each student without redundancy!

Because of the one-to-many relationship between students and subjects, it's more natural to structure this as a dictionary structure, which is equivalent to the JSON format that NoSQL uses:

In [39]:
student_subjects = dict(zip(students, subjects))
student_subjects

{'Aysja Johnson': ['Semi-supervised learning'],
 'Gabriel Augusto Sanchez': ['Time series analysis'],
 'Ivan Dimitrov': ['Network analysis', 'Social networks'],
 'Jimin Yu': ['Association rules'],
 'Karen Leung': ['Shiny'],
 'Kurt Brown': ['Bayesian methods',
  'Multi-level modeling',
  'Hierarchical methods'],
 'Matt Bartholomew': ['Apache Spark'],
 'Nathan Helm Burger': ['MapReduce', 'Apache Hadoop'],
 'Rafael Lacerda': ['SQL vs. NoSQL'],
 'Sven Chilton': ['Support vector machines'],
 'Tegan McCaslin': ['Markov models', 'Hidden Markov models'],
 'Zi (Russ) Gu': ['Multiple hypothesis testing',
  'Bonferroni correction',
  'A/B testing',
  'Multi-armed bandit']}

To add information about subjects to our NoSQL students entity, we must iterate through each one individually and add the new data:

In [106]:
for row in nosql_students.all():
    # We cannot update the UnQLite dictionary directly:
    row_dict = row.copy() 
    name = row_dict['name']
    # We need to check if a certain student is not on
    # the subjects list to avoid a dictionary error.
    if name in student_subjects.keys():
        subjects_list = student_subjects[name]
        subjects_dict = {'subjects':subjects_list}
        row_dict.update(subjects_dict)
        nosql_students.update(
            row['__id'],
            row_dict
        )

Notice from the element below how data is stored in the database, in a very similar format to a JSON node or Python dictionary - which incidentally is that structure that Python NoSQL APIs use to represent elements.
Contrasting against SQL, instead of rows NoSQL has documents, which can be tree-like structures.

In [107]:
# Take a look at how NoSQL structures this data:
nosql_students.fetch(2)

{'__id': 2,
 'email': 'ivan@emailio.com',
 'name': 'Ivan Dimitrov',
 'subjects': ['Network analysis', 'Social networks']}

Even though our data has a tree-like structure, the Pandas DataFrame object below still manages to represent multiple items within a single cell. Also note that while the dataframe shows NaNs for values that weren't specified, these NaN's aren't actually stored in the database, as we can see from the output of <code>all()</code>.

In [108]:
# Also look at how this is represented in a Pandas dataframe:
pd.DataFrame(nosql_students.all())

__id                         email                     name         phone  \
0      0        aysjajohnson@gmail.com            Aysja Johnson           NaN   
1      1          gebsanchez@gmail.com  Gabriel Augusto Sanchez           NaN   
2      2              ivan@emailio.com            Ivan Dimitrov           NaN   
3      3                jy1003@nyu.edu                 Jimin Yu           NaN   
4      4            kleung92@gmail.com              Karen Leung           NaN   
5      5        kurtbrown137@gmail.com               Kurt Brown           NaN   
6      6        bartholomatt@gmail.com         Matt Bartholomew           NaN   
7      7  nathan.helm.burger@gmail.com       Nathan Helm Burger           NaN   
8      8        rafaelplmelo@gmail.com           Rafael Lacerda           NaN   
9      9             rustygu@gmail.com             Zi (Russ) Gu           NaN   
10    10        sven.chilton@gmail.com             Sven Chilton           NaN   
11    11      tegan.mccaslin@gmail.com           Tegan McCaslin           NaN   
12    12        mcnamerson@example.com         Namey McNamerson  555 555 5555   

                                             subjects  
0                          [Semi-supervised learning]  
1                              [Time series analysis]  
2                 [Network analysis, Social networks]  
3                                 [Association rules]  
4                                             [Shiny]  
5   [Bayesian methods, Multi-level modeling, Hiera...  
6                                      [Apache Spark]  
7                          [MapReduce, Apache Hadoop]  
8                                     [SQL vs. NoSQL]  
9   [Multiple hypothesis testing, Bonferroni corre...  
10                          [Support vector machines]  
11              [Markov models, Hidden Markov models]  
12                                                NaN

It's quite clear that NoSQL deals with unstructured data in a more flexible and efficient way than MySQL.

Using this approach, we can keep all the data in a single entity. The advantage of this approach is that the data can be used as needed without having to specify Joins when creating queries.

Even though we managed to save space while avoiding normalization, there still is a strong argument to support it:
In our application, we must consider how often we need to query only for subjects versus how often we need to query only for contact information, separately: If every time we query for contact info we also receive a list of subjects, this is becomes a source for bandwidth inefficiency! There are no fixed rules for this and the "distance" between data needs to be considered when normalizing databases.

### Transactions and data integrity

Suppose that while we were updating the unnormalized NoSQL student's object to add subjects, we forgot to check if the student existed in the subjects list:

We would get an error when the student wasn't found, part of the data would be updated and the other part wouldn't, leaving us with a data integrity issue to solve!

This however, would not happen in SQL, especially if using *transactions*.

SQL's transactions conform to a standard known by the acronym ACID:

- **Atomicity**: Either all parts of a transaction must be completed or none.
- **Consistency**: The integrity of the database is preserved by all transactions. The database is not left in an invalid state after a transaction.
- **Isolation**: A transaction must be run isolated in order to guarantee that any inconsistency in the data involved does not affect other transactions.
- **Durability**: The changes made by a completed transaction must be preserved or inother words be durable.

<sub>SOURCE: 2013, HADJIGEORGIOU, C. in "RDBMS vs NoSQL: Performance and Scaling Comparison" p. 4, available at https://static.ph.ed.ac.uk/dissertations/hpc-msc/2012-2013/RDBMS%20vs%20NoSQL%20-%20Performance%20and%20Scaling%20Comparison.pdf </sub>

In a first degree, SQL only inserts, deletes or updates data in a table if all row operations are successful. In a second degree, if using transactions, SQL may receive several commands, altering several tables and will only save the changes if all operations are successful and we call the <code>commit()</code> function. If any errors are made, a <code>rollback()</code> call will restore the database state back to the start of our transaction.

To trigger an error, let's attempt to create a subject for a student that doesn't exist - that is, a subject with an invalid Foreign Key: 

In [43]:
# With SQLite, transactions (including initializing one,
# commiting or rolling back) are implicit. We only need
# to use the "with" structure with the connection object.

with connection as conn:
    cursor = conn.cursor()
    try:
        cursor.execute(""" 
                        INSERT INTO subjects(subject, student_id)
                        VALUES('Flux Capacitor Learning', 34); 
                        """)
    except sql.IntegrityError as err:
        print(err) # This way we can actually see what went wrong.

FOREIGN KEY constraint failed


As expected, SQL denies the entire operation because we set a constraint that a subject must be attached to a student that actually exists.
By querying the subjects table, we can verify that nothing has changed, preserving the integrity of our database:

In [27]:
with connection as conn:
    cursor = conn.cursor()
    cursor.execute("""
                SELECT a.name, a.email, b.subject
                FROM students AS a
                LEFT JOIN subjects AS b
                ON a.id == b.student_id
                ;
                """)

sql_subjects = cursor.fetchall()
sql_subjects_df = pd.DataFrame(
    data=sql_subjects,
    columns=[i[0] for i in cursor.description]
)
sql_subjects_df

name                         email  \
0             Aysja Johnson        aysjajohnson@gmail.com   
1             Aysja Johnson        aysjajohnson@gmail.com   
2   Gabriel Augusto Sanchez          gebsanchez@gmail.com   
3             Ivan Dimitrov              ivan@emailio.com   
4             Ivan Dimitrov              ivan@emailio.com   
5                  Jimin Yu                jy1003@nyu.edu   
6               Karen Leung            kleung92@gmail.com   
7                Kurt Brown        kurtbrown137@gmail.com   
8                Kurt Brown        kurtbrown137@gmail.com   
9                Kurt Brown        kurtbrown137@gmail.com   
10         Matt Bartholomew        bartholomatt@gmail.com   
11       Nathan Helm Burger  nathan.helm.burger@gmail.com   
12       Nathan Helm Burger  nathan.helm.burger@gmail.com   
13           Rafael Lacerda        rafaelplmelo@gmail.com   
14             Zi (Russ) Gu             rustygu@gmail.com   
15             Zi (Russ) Gu             rustygu@gmail.com   
16             Zi (Russ) Gu             rustygu@gmail.com   
17             Zi (Russ) Gu             rustygu@gmail.com   
18             Sven Chilton        sven.chilton@gmail.com   
19           Tegan McCaslin      tegan.mccaslin@gmail.com   
20           Tegan McCaslin      tegan.mccaslin@gmail.com   
21         Namey McNamerson        mcnamerson@example.com   

                        subject  
0      Semi-supervised learning  
1      Semi-supervised learning  
2          Time series analysis  
3              Network analysis  
4               Social networks  
5             Association rules  
6                         Shiny  
7              Bayesian methods  
8          Hierarchical methods  
9          Multi-level modeling  
10                 Apache Spark  
11                Apache Hadoop  
12                    MapReduce  
13                SQL vs. NoSQL  
14                  A/B testing  
15        Bonferroni correction  
16           Multi-armed bandit  
17  Multiple hypothesis testing  
18      Support vector machines  
19         Hidden Markov models  
20                Markov models  
21                         None

On matter of data integrity it's clear that SQL has the advantage. While it is good practice to validate data within the application before feeding it into the database, NoSQL will require extra work in this aspect.

### Use Cases

Having seen the differences between MySQL and NoSQL, we can conclude that the best choice is highly project dependent:

- For projects where the data model is well defined, SQL's formal structure can stop you from making mistakes.
- For an evolving data model, NoSQL's flexibility allows you to move fast.
- If data integrity is essential, SQL's all-or-nothing transactions are a safety net.

<sub>SOURCE: https://www.sitepoint.com/sql-vs-nosql-differences/</sub>